In [5]:
import pandas as pd
import os

In [147]:
def import_csvs(folder):
    ret = {}
    for f in filter(lambda x: ".csv" in x.lower(), os.listdir(folder)):
        ret[f] = pd.read_csv(folder + f)
    return ret

state_frames = import_csvs("data/")
national_frames = import_csvs("data/national/")
def tsvframe(df):
    return df.to_csv(index=False,sep="\t")

In [7]:
def collapse_all_but(df, cols):
    masks = [("SEX.display-label","All firms"),
             ("RACE_GROUP.display-label","All firms"),
             ("ETH_GROUP.display-label","All firms"),
             ("VET_GROUP.display-label","All firms"),
             ("YIBSZFI.display-label","All firms"),
             ("NAICS.display-label","Total for all sectors")]
    tmpdf = df;
    for mask in masks:
        if mask[0] in cols:
            continue
        tmpdf = tmpdf[tmpdf[mask[0]] == mask[1]]
    return tmpdf

def collapse_race(df):
    return collapse_all_but(df, ["RACE_GROUP.display-label"])

def collapse(df):
    return collapse_all_but(df, ["NOTHING"])

In [9]:
state_frames.keys()

['ASE_2014_00CSA04_with_ann.csv',
 'ASE_2014_00CSA03_with_ann.csv',
 'ASE_2014_00CSA01_with_ann.csv',
 'ASE_2014_00CSA01_metadata.csv',
 'ASE_2014_00CSA02_metadata.csv',
 'ASE_2014_00CSA04_metadata.csv',
 'ASE_2014_00CSA02_with_ann.csv',
 'ASE_2014_00CSA03_metadata.csv']

In [56]:
state_frames["ASE_2014_00CSA02_metadata.csv"]

,GEO.id,Geographic identifier code
0,GEO.id2,Id2
1,GEO.display-label,Geographic area name
2,NAICS.id,2012 NAICS code
3,NAICS.display-label,Meaning of 2012 NAICS code
4,SEX.id,Gender code
5,SEX.display-label,Meaning of Gender code
6,ETH_GROUP.id,Ethnicity code
7,ETH_GROUP.display-label,Meaning of Ethnicity code
8,RACE_GROUP.id,Race code
9,RACE_GROUP.display-label,Meaning of Race code


In [130]:
cols = "all","11-15","16+","2-3","4-5","6-10","<2"
def piv(col):
    df = state_frames["ASE_2014_00CSA02_with_ann.csv"]
    df = df[["GEO.display-label","YIBSZFI.display-label",col]].drop(0)

#df = pd.pivot_table(df, index="GEO.display-label",
#                   columns="YIBSZFI.display-label",
#                   values="PAYANN")
    df[col] = pd.to_numeric(df[col],errors="coerce")
    df = pd.pivot_table(df, index="GEO.display-label",
              columns="YIBSZFI.display-label",
              values=col)
    df.columns = cols
    df["0-10"] = df["<2"] + df["2-3"] + df["4-5"] + df["6-10"]
    df["10+"] = df["11-15"] + df["16+"]

    return df
    
df = piv("PAYANN").join(piv("EMP"),rsuffix="_r")

def div(x, col):
    return x[col] / x[col + "_r"]

def conv(df, col):
    df[col + "_div"] = df.apply(lambda x: div(x, col),axis=1)
    return df

for col in cols:
    conv(df, col)
    
conv(df,"0-10")
conv(df,"10+")
df_10 = df.reset_index()
df = df[map(lambda x: x + "_div",cols)][[
        "all_div",
        "<2_div",
        "2-3_div",
        "4-5_div",
        "6-10_div",
        "11-15_div",
        "16+_div"]].sort_values(by="all_div",ascending=False).reset_index()

In [131]:
df.sort_values(by="all_div",ascending=False).reset_index()

,index,GEO.display-label,all_div,<2_div,2-3_div,4-5_div,6-10_div,11-15_div,16+_div
0,0,District of Columbia,69.680771,45.668668,53.484140,59.661102,49.297443,71.094467,74.761622
1,1,Massachusetts,57.674920,32.155909,39.531320,38.461952,43.698748,55.851845,65.508289
2,2,New York,57.637198,33.870655,33.721826,38.493747,39.287410,52.066411,75.025651
3,3,New Jersey,53.331920,27.749954,29.436944,32.432198,39.270089,51.862635,65.961186
4,4,Connecticut,53.243737,27.932573,28.037941,30.437688,43.409730,48.694380,63.290192
5,5,Alaska,53.013817,36.526225,43.680470,36.227380,42.168253,52.435335,61.604489
6,6,California,52.960231,32.135933,32.121473,37.624019,46.672625,49.112638,63.952713
7,7,Washington,49.410306,28.186664,30.229415,40.032100,37.144499,42.010804,63.473797
8,8,Delaware,48.844796,39.037495,28.238252,22.636532,33.573104,47.619179,56.562005
9,9,Colorado,48.713960,30.067991,32.542833,34.161391,38.012755,44.859131,60.450069


In [162]:
payroll_chart = df[df["GEO.display-label"] == "Connecticut"].transpose().reset_index().drop(0)

payroll_chart[4] = payroll_chart.apply(lambda x: x[4] * 1000, axis=1)
print tsvframe(payroll_chart.round())

index	4
all_div	53244.0
<2_div	27933.0
2-3_div	28038.0
4-5_div	30438.0
6-10_div	43410.0
11-15_div	48694.0
16+_div	63290.0



In [132]:
df.sort_values(by="<2_div",ascending=False).reset_index()

,index,GEO.display-label,all_div,<2_div,2-3_div,4-5_div,6-10_div,11-15_div,16+_div
0,17,North Dakota,45.104169,49.190937,46.219496,41.715505,32.138608,41.951676,51.115654
1,0,District of Columbia,69.680771,45.668668,53.484140,59.661102,49.297443,71.094467,74.761622
2,18,Rhode Island,44.192980,41.917602,25.914052,24.140320,38.008057,43.187645,51.816956
3,38,Nevada,38.154631,40.321845,22.964550,31.070664,38.877869,41.622809,39.550559
4,46,West Virginia,35.988345,40.242692,26.819399,33.096442,29.357062,31.416560,41.026283
5,8,Delaware,48.844796,39.037495,28.238252,22.636532,33.573104,47.619179,56.562005
6,5,Alaska,53.013817,36.526225,43.680470,36.227380,42.168253,52.435335,61.604489
7,27,Oklahoma,40.876715,35.070791,32.165398,31.324042,36.560686,39.959349,45.594218
8,12,Texas,47.423725,34.892763,34.311377,33.973045,38.031512,45.752073,55.063845
9,2,New York,57.637198,33.870655,33.721826,38.493747,39.287410,52.066411,75.025651


In [133]:
df.sort_values(by="2-3_div",ascending=False).reset_index()

,index,GEO.display-label,all_div,<2_div,2-3_div,4-5_div,6-10_div,11-15_div,16+_div
0,0,District of Columbia,69.680771,45.668668,53.484140,59.661102,49.297443,71.094467,74.761622
1,17,North Dakota,45.104169,49.190937,46.219496,41.715505,32.138608,41.951676,51.115654
2,5,Alaska,53.013817,36.526225,43.680470,36.227380,42.168253,52.435335,61.604489
3,1,Massachusetts,57.674920,32.155909,39.531320,38.461952,43.698748,55.851845,65.508289
4,12,Texas,47.423725,34.892763,34.311377,33.973045,38.031512,45.752073,55.063845
5,11,Illinois,47.847649,32.540480,34.147413,33.606401,41.862522,46.109386,52.595779
6,2,New York,57.637198,33.870655,33.721826,38.493747,39.287410,52.066411,75.025651
7,10,Maryland,48.482069,30.013402,33.017617,36.200667,41.395260,47.872490,54.646132
8,9,Colorado,48.713960,30.067991,32.542833,34.161391,38.012755,44.859131,60.450069
9,27,Oklahoma,40.876715,35.070791,32.165398,31.324042,36.560686,39.959349,45.594218


In [134]:
df.sort_values(by="4-5_div",ascending=False).reset_index()

,index,GEO.display-label,all_div,<2_div,2-3_div,4-5_div,6-10_div,11-15_div,16+_div
0,0,District of Columbia,69.680771,45.668668,53.484140,59.661102,49.297443,71.094467,74.761622
1,17,North Dakota,45.104169,49.190937,46.219496,41.715505,32.138608,41.951676,51.115654
2,7,Washington,49.410306,28.186664,30.229415,40.032100,37.144499,42.010804,63.473797
3,2,New York,57.637198,33.870655,33.721826,38.493747,39.287410,52.066411,75.025651
4,1,Massachusetts,57.674920,32.155909,39.531320,38.461952,43.698748,55.851845,65.508289
5,6,California,52.960231,32.135933,32.121473,37.624019,46.672625,49.112638,63.952713
6,13,Virginia,47.110183,26.616647,30.505199,36.368074,42.673330,45.316257,54.682236
7,5,Alaska,53.013817,36.526225,43.680470,36.227380,42.168253,52.435335,61.604489
8,10,Maryland,48.482069,30.013402,33.017617,36.200667,41.395260,47.872490,54.646132
9,15,Wyoming,45.204621,30.617422,29.955337,34.810601,38.645052,41.894637,55.695631


In [135]:
df.sort_values(by="6-10_div",ascending=False).reset_index()

,index,GEO.display-label,all_div,<2_div,2-3_div,4-5_div,6-10_div,11-15_div,16+_div
0,0,District of Columbia,69.680771,45.668668,53.484140,59.661102,49.297443,71.094467,74.761622
1,6,California,52.960231,32.135933,32.121473,37.624019,46.672625,49.112638,63.952713
2,1,Massachusetts,57.674920,32.155909,39.531320,38.461952,43.698748,55.851845,65.508289
3,4,Connecticut,53.243737,27.932573,28.037941,30.437688,43.409730,48.694380,63.290192
4,13,Virginia,47.110183,26.616647,30.505199,36.368074,42.673330,45.316257,54.682236
5,5,Alaska,53.013817,36.526225,43.680470,36.227380,42.168253,52.435335,61.604489
6,11,Illinois,47.847649,32.540480,34.147413,33.606401,41.862522,46.109386,52.595779
7,10,Maryland,48.482069,30.013402,33.017617,36.200667,41.395260,47.872490,54.646132
8,2,New York,57.637198,33.870655,33.721826,38.493747,39.287410,52.066411,75.025651
9,3,New Jersey,53.331920,27.749954,29.436944,32.432198,39.270089,51.862635,65.961186


In [136]:
df.sort_values(by="11-15_div",ascending=False).reset_index()

,index,GEO.display-label,all_div,<2_div,2-3_div,4-5_div,6-10_div,11-15_div,16+_div
0,0,District of Columbia,69.680771,45.668668,53.484140,59.661102,49.297443,71.094467,74.761622
1,1,Massachusetts,57.674920,32.155909,39.531320,38.461952,43.698748,55.851845,65.508289
2,5,Alaska,53.013817,36.526225,43.680470,36.227380,42.168253,52.435335,61.604489
3,2,New York,57.637198,33.870655,33.721826,38.493747,39.287410,52.066411,75.025651
4,3,New Jersey,53.331920,27.749954,29.436944,32.432198,39.270089,51.862635,65.961186
5,6,California,52.960231,32.135933,32.121473,37.624019,46.672625,49.112638,63.952713
6,4,Connecticut,53.243737,27.932573,28.037941,30.437688,43.409730,48.694380,63.290192
7,10,Maryland,48.482069,30.013402,33.017617,36.200667,41.395260,47.872490,54.646132
8,8,Delaware,48.844796,39.037495,28.238252,22.636532,33.573104,47.619179,56.562005
9,11,Illinois,47.847649,32.540480,34.147413,33.606401,41.862522,46.109386,52.595779


In [137]:
df.sort_values(by="16+_div",ascending=False).reset_index()

,index,GEO.display-label,all_div,<2_div,2-3_div,4-5_div,6-10_div,11-15_div,16+_div
0,2,New York,57.637198,33.870655,33.721826,38.493747,39.287410,52.066411,75.025651
1,0,District of Columbia,69.680771,45.668668,53.484140,59.661102,49.297443,71.094467,74.761622
2,3,New Jersey,53.331920,27.749954,29.436944,32.432198,39.270089,51.862635,65.961186
3,1,Massachusetts,57.674920,32.155909,39.531320,38.461952,43.698748,55.851845,65.508289
4,6,California,52.960231,32.135933,32.121473,37.624019,46.672625,49.112638,63.952713
5,7,Washington,49.410306,28.186664,30.229415,40.032100,37.144499,42.010804,63.473797
6,4,Connecticut,53.243737,27.932573,28.037941,30.437688,43.409730,48.694380,63.290192
7,5,Alaska,53.013817,36.526225,43.680470,36.227380,42.168253,52.435335,61.604489
8,9,Colorado,48.713960,30.067991,32.542833,34.161391,38.012755,44.859131,60.450069
9,8,Delaware,48.844796,39.037495,28.238252,22.636532,33.573104,47.619179,56.562005


In [139]:
df_10[["GEO.display-label","0-10_div","10+_div"]].sort_values(by="0-10_div",ascending=False)

,GEO.display-label,0-10_div,10+_div
8,District of Columbia,53.501194,73.224832
1,Alaska,41.057566,57.183679
21,Massachusetts,40.477550,60.933532
4,California,39.964772,57.428330
34,North Dakota,39.763511,46.535813
13,Illinois,37.623174,49.537285
46,Virginia,37.397030,50.144279
32,New York,37.242757,63.600564
20,Maryland,37.037314,51.221268
43,Texas,35.965098,51.401790


In [140]:
df_10[["GEO.display-label","0-10_div","10+_div"]].sort_values(by="10+_div",ascending=False)

,GEO.display-label,0-10_div,10+_div
8,District of Columbia,53.501194,73.224832
32,New York,37.242757,63.600564
21,Massachusetts,40.477550,60.933532
30,New Jersey,34.365232,58.457412
4,California,39.964772,57.428330
1,Alaska,41.057566,57.183679
6,Connecticut,35.387774,56.128590
47,Washington,35.185496,53.250204
7,Delaware,32.372534,53.209733
5,Colorado,35.193001,53.201637
